### Cleaning the migration dataset through SQL

In [1]:
# Python SQL toolkit and Object Relational Mapper
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import json

### Store CSV into DataFrame
Migration dataset source: https://www.kaggle.com/finnegannguyen/statetostate-migration-flows-from-2010-to-2019
From there, grab unique current states and get lat and lon through a gmap request

In [2]:
csv_file_state = "Resources/data/state_coord.csv"
state_coord = pd.read_csv(csv_file_state)
state_coord.head()

,state,lat,lng
0,Alabama,32.318231,-86.902298
1,Alaska,64.200841,-149.493673
2,Arizona,34.048928,-111.093731
3,Arkansas,35.201050,-91.831833
4,California,36.778261,-119.417932


In [3]:
# import the migration file
migration_file = "Resources/data/data_raw.csv"
migration_df = pd.read_csv(migration_file)
migration_df.head()

,current_state,year,population,same_house,same_state,from_different_state_Total,abroad_Total,from,number_of_people
0,Alabama,2010,4729509,3987155,620465,108723,13166,Alabama,0
1,Alaska,2010,702974,565031,95878,36326,5739,Alabama,477
2,Arizona,2010,6332786,5069002,1001991,222725,39068,Alabama,416
3,Arkansas,2010,2888304,2387806,412997,79127,8374,Alabama,1405
4,California,2010,36907897,30790221,5413287,444749,259640,Alabama,3364


In [4]:
# get only the columns needed
migration_df = migration_df[['current_state', 'year', 'population','from','number_of_people']].copy()

In [5]:
# rename the columns
migration_df = migration_df.rename(columns={"current_state": "destination", "from": "origin", "number_of_people": "total"})
migration_df.head()

,destination,year,population,origin,total
0,Alabama,2010,4729509,Alabama,0
1,Alaska,2010,702974,Alabama,477
2,Arizona,2010,6332786,Alabama,416
3,Arkansas,2010,2888304,Alabama,1405
4,California,2010,36907897,Alabama,3364


In [6]:
# find the unique states, to ensure that there is a match with foreign key
migration_df['origin'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia ',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'abroad_PuertoRico',
       'abroad_USIslandArea', 'abroad_ForeignCountry'], dtype=object)

In [7]:
#include only list where state has coordinates from the other csv file
inc_list = state_coord['state'].tolist()

In [8]:
migration_df = migration_df[migration_df['origin'].isin(inc_list)]
migration_df['origin'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia ',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [9]:
postal_data = "Resources/data/postal_data.csv"
postal_df = pd.read_csv(postal_data)
postal_df.head()

,state,postal
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


### Connect to local database

In [10]:
rds_connection_string = "postgres:postgres@localhost:5432/project_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [11]:
# Confirm tables
engine.table_names()

<ipython-input-11-2d97e9123e98>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['migration_table', 'states_coord', 'postal_data']

### Load the Datasets

In [12]:
state_coord.to_sql(name='states_coord', con=engine, if_exists='append', index=False)

In [13]:
migration_df.to_sql(name='migration_table', con=engine, if_exists='append', index=False)

In [14]:
postal_df.to_sql(name='postal_data', con=engine, if_exists='append', index=False)

### Confirm data has been added

In [15]:
sql_one = pd.read_sql_query('select * from states_coord', con=engine)
sql_one = pd.DataFrame(sql_one)

In [16]:
sql_two = pd.read_sql_query('select * from migration_table', con=engine)
sql_two = pd.DataFrame(sql_two)

In [17]:
sql_three = pd.read_sql_query('select * from postal_data', con=engine)
sql_three = pd.DataFrame(sql_three)


### Use sql data to clean and filter in jupyter

In [18]:
migration_merged = sql_two.merge(sql_one, left_on='destination', right_on='state',
          suffixes=('_left', '_right'))
migration_merged = migration_merged.drop(['state'], axis=1)
migration_merged = migration_merged.rename(columns={"lat": "destination_lat", "lng": "destination_lng"})

In [19]:
migration_merged = migration_merged.merge(sql_one, left_on='origin', right_on='state',
          suffixes=('_left', '_right'))
migration_merged = migration_merged.drop(['state'], axis=1)
migration_merged = migration_merged.rename(columns={"lat": "origin_lat", "lng": "origin_lng"})

In [20]:
# migration_merged = migration_merged.re_index()
migration_merged.head()

,destination,year,population,origin,total,destination_lat,destination_lng,origin_lat,origin_lng
0,Alabama,2010,4729509,Alabama,0,32,-87,32,-87
1,Alabama,2011,4745278,Alabama,0,32,-87,32,-87
2,Alabama,2012,4764428,Alabama,0,32,-87,32,-87
3,Alabama,2013,4781296,Alabama,0,32,-87,32,-87
4,Alabama,2014,4791931,Alabama,0,32,-87,32,-87


In [21]:
migration_merged.to_csv("Resources/data/migration_merged.csv",index=False)

In [23]:
state_merged = migration_merged.merge(sql_three, left_on='destination', right_on='state',
          suffixes=('_left', '_right'))
state_merged = state_merged.drop(['state'], axis=1)
state_merged.head()

,destination,year,population,origin,total,destination_lat,destination_lng,origin_lat,origin_lng,postal
0,Alabama,2010,4729509,Alabama,0,32,-87,32,-87,AL
1,Alabama,2011,4745278,Alabama,0,32,-87,32,-87,AL
2,Alabama,2012,4764428,Alabama,0,32,-87,32,-87,AL
3,Alabama,2013,4781296,Alabama,0,32,-87,32,-87,AL
4,Alabama,2014,4791931,Alabama,0,32,-87,32,-87,AL


In [24]:
# state_merged = state_merged.groupby(['destination','postal','year']).sum()
# state_merged.head()

In [25]:
# state_merged = state_merged.reset_index()
state_merged = state_merged[['destination','year','total','postal','origin']]

In [26]:
state_merged = state_merged.groupby(['destination','year']).sum().groupby(level=0).cumsum().reset_index()

In [30]:
state_merged.head()

,destination,year,total
0,Alabama,2010,108723
1,Alabama,2011,226449
2,Alabama,2012,331049
3,Alabama,2013,435151
4,Alabama,2014,543070


In [35]:
state_merged['pct_ch'] = (state_merged.groupby('destination')['total']
                                  .apply(pd.Series.pct_change) + 1)

In [36]:
state_merged.head(30)

,destination,year,total,pct_ch
0,Alabama,2010,108723,NaN
1,Alabama,2011,226449,2.082807
2,Alabama,2012,331049,1.461914
3,Alabama,2013,435151,1.314461
4,Alabama,2014,543070,1.248004
5,Alabama,2015,657946,1.211531
6,Alabama,2016,780166,1.185760
7,Alabama,2017,891969,1.143307
8,Alabama,2018,1001270,1.122539
9,Alabama,2019,1106050,1.104647


In [42]:
state_merged = state_merged.merge(sql_three, left_on='destination', right_on='state',
          suffixes=('_left', '_right'))
state_merged = state_merged.drop(['state'], axis=1)
state_merged.head(50)

,destination,year,total,pct_ch,postal_left,postal_right
0,Alabama,2010,108723,NaN,AL,AL
1,Alabama,2011,226449,2.082807,AL,AL
2,Alabama,2012,331049,1.461914,AL,AL
3,Alabama,2013,435151,1.314461,AL,AL
4,Alabama,2014,543070,1.248004,AL,AL
5,Alabama,2015,657946,1.211531,AL,AL
6,Alabama,2016,780166,1.185760,AL,AL
7,Alabama,2017,891969,1.143307,AL,AL
8,Alabama,2018,1001270,1.122539,AL,AL
9,Alabama,2019,1106050,1.104647,AL,AL


In [38]:
state_merged.to_csv("Resources/data/state_merged.csv",index=True)

In [41]:
state_merged['pct_ch'].max()

2.264056148855058

In [ ]:
grouped_migration = migration_merged[['year','total']]
grouped_migration = grouped_migration.groupby(['year']).sum()
grouped_migration = grouped_migration.reset_index()

In [ ]:
grouped_migration

In [ ]:
difference = grouped_migration.diff()
difference

In [ ]:
change = pd.DataFrame((difference['total']/grouped_migration['total'])*100)
change = change.rename(columns={"total": "change"})
change = change.cumsum()

In [ ]:
grouped_migration = pd.concat([grouped_migration, change], axis=1)

In [ ]:
grouped_migration

In [ ]:
grouped_migration.to_json("Resources/data/grouped_migration.json")